# Alzheimer’s Disease Pathway — Min-Cut Pipeline

What this notebook does

- 1. Fetch KEGG hsa05010 (Alzheimer’s disease) genes via KEGG REST API
- 2. Map to gene symbols; query STRING API for PPI network (human 9606)
- 3. Build a weighted NetworkX graph
- 4. Compute:
  - Global min-cut (Stoer–Wagner, weighted)
  - s–t min edge cut (sources: APP/PSEN1/PSEN2/MAPT; sinks: CASP3/CASP8/APAF1)
  - Karger randomized min-cut on unweighted graph (10/100/1000 trials)

5.  Visualize network snapshot (cut edges) and a trials-vs-average plot


In [ ]:
import io
import sys
import math
import time
from collections import defaultdict

import requests
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

print(sys.version)
print("NetworkX:", nx.__version__)

In [ ]:
KEGG_BASE = "http://rest.kegg.jp"
KEGG_PATHWAY_ID = "hsa05010"  # Alzheimer's disease

def kegg_link_hsa(pathway_id: str) -> pd.DataFrame:
    """
    KEGG: link hsa <pathway> → pathway-to-gene rows: path:hsa05010 \t hsa:351
    """
    url = f"{KEGG_BASE}/link/hsa/{pathway_id}"
    r = requests.get(url, timeout=30)
    r.raise_for_status()
    rows = [line.split("\t") for line in r.text.strip().splitlines()]
    df = pd.DataFrame(rows, columns=["pathway", "hsa_id"])
    return df

def kegg_get_gene_symbols(hsa_ids: list) -> pd.DataFrame:
    """
    KEGG: get hsa:XXXX blocks; parse NAME line for the primary gene symbol.
    """
    chunks = []
    BATCH = 50
    for i in range(0, len(hsa_ids), BATCH):
        batch = hsa_ids[i:i+BATCH]
        ids_str = "+".join(batch)
        url = f"{KEGG_BASE}/get/{ids_str}"
        r = requests.get(url, timeout=60)
        r.raise_for_status()
        # Split into entries using 'ENTRY' as delimiter
        entries = r.text.split("ENTRY")
        for ent in entries:
            if not ent.strip():
                continue
            # Try to find an 'hsa:' token in the block (gene id)
            tokens = [t for t in ent.split() if t.startswith("hsa:")]
            hsa_id = tokens[0] if tokens else None

            # Parse NAME line: e.g., "NAME        APP; amyloid beta precursor protein"
            symbol = None
            for ln in ent.splitlines():
                if ln.strip().startswith("NAME"):
                    rest = ln.split("NAME", 1)[1].strip()
                    if ";" in rest:
                        symbol = rest.split(";")[0].strip()
                    else:
                        symbol = rest.split(",")[0].strip()
                    break
            if hsa_id and symbol:
                chunks.append((hsa_id, symbol))
    return pd.DataFrame(chunks, columns=["hsa_id", "gene_symbol"])

# Fetch KEGG genes
link_df = kegg_link_hsa(KEGG_PATHWAY_ID)
hsa_ids = sorted(link_df["hsa_id"].unique().tolist())
genes_df = kegg_get_gene_symbols(hsa_ids)
print("Pathway:", KEGG_PATHWAY_ID, "| genes:", len(genes_df))
display(genes_df.head())


In [ ]:
STRING_BASE = "https://string-db.org/api"
STRING_FORMAT = "tsv"
STRING_METHOD = "network"
SPECIES = 9606
SCORE_CUTOFF = 700  # high confidence (0..1000)

def string_network_from_genes(genes: list, species: int = 9606, score_cutoff: int = 700) -> pd.DataFrame:
    """
    STRING 'tsv/network' API: pass identifiers via query (URL-encoded newline %0d).
    Returns a TSV where preferredName_A/B and score (combined) are present.
    """
    # STRING supports identifiers separated by '%0d' (CR). Works fine for symbols.
    identifiers = "%0d".join(genes)
    url = f"{STRING_BASE}/{STRING_FORMAT}/{STRING_METHOD}?identifiers={identifiers}&species={species}&required_score={score_cutoff}"
    r = requests.get(url, timeout=90)
    r.raise_for_status()
    df = pd.read_csv(io.StringIO(r.text), sep="\t")
    return df

gene_list = genes_df["gene_symbol"].dropna().unique().tolist()
str_df = string_network_from_genes(gene_list, species=SPECIES, score_cutoff=SCORE_CUTOFF)
print(f"STRING edges (score ≥ {SCORE_CUTOFF}):", len(str_df))
display(str_df.head())


In [ ]:
def build_graph_from_string(df: pd.DataFrame, allowed_nodes: set) -> nx.Graph:
    """
    Build undirected weighted graph from STRING network table.
    Uses preferredName_A/B (or protein1/protein2 fallback) and score as weight.
    """
    G = nx.Graph()
    for _, row in df.iterrows():
        a = row.get("preferredName_A") or row.get("protein1")
        b = row.get("preferredName_B") or row.get("protein2")
        if not a or not b:
            continue
        if a not in allowed_nodes or b not in allowed_nodes:
            continue
        w = int(row.get("score") or row.get("combined_score") or 0)
        if G.has_edge(a, b):
            # keep max weight if duplicates appear
            if w > G[a][b].get("weight", 0):
                G[a][b]["weight"] = w
        else:
            G.add_edge(a, b, weight=w)
    return G

allowed = set(gene_list)
G = build_graph_from_string(str_df, allowed)
print("Nodes:", G.number_of_nodes(), "| Edges:", G.number_of_edges())

# Basic sanity: connected components
comps = sorted(nx.connected_components(G), key=len, reverse=True)
print("Connected components:", len(comps), "| largest size:", len(comps[0]) if comps else 0)
